<a href="https://colab.research.google.com/github/beingraja/visual_question_answering/blob/main/use_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 1-CsnNc63DBXn77XyJ15da-Rf0eRR6rr-
!gdown --id 1zcf2xN_oNSUoK3y3vTG6nFPyolAXxYW9

Downloading...
From: https://drive.google.com/uc?id=1-CsnNc63DBXn77XyJ15da-Rf0eRR6rr-
To: /content/model.h5
2.63GB [00:32, 80.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zcf2xN_oNSUoK3y3vTG6nFPyolAXxYW9
To: /content/ans_dict_inv.pickle
100% 220k/220k [00:00<00:00, 31.8MB/s]


In [ ]:
!pip install tensorflow
!pip install tensorflow_hub
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 51kB 4.0MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-cp37-none-any.whl size=30535 sha256=6ab545c1609a9d9e9641bbb273ebbccbce9a68c6e4ab0d3580072da3e1fdc3aa
  Stored in directory: /root/.cache/pip/wheels/a1/04/ee/347bd9f5b821b637c76411d280271a857aece00358896a230f
  Created wheel for py-params: filename=py_params-0.10.2-cp37-none-any.whl size=7912 sha256=05d7bda9ebe5ba3abfe9b52608f93c22275b6bedfc072c03d7713e377757ff23
  Stored in directory: /root/.cache/pip/wheels/d0/4a/70/ff12450229ff1955abf01f365051d4faae1c20aef53ab4cf09
  Created wheel for params-flow: filename=params_flow-0.8.2-cp37-none-any.whl size=19472 sha256=a961da547c7db7bb684a9229e1e33e85d98094872d2a0150d896bf8ec0f5fcb4
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.2MB 5.1MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
 
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)
 
from bert import bert_tokenization
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
 trainable=True)
BertTokenizer = bert_tokenization.FullTokenizer
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer =  bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

TF version:  2.5.0
Hub version:  0.12.0


In [ ]:
def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))
 
 
def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))
 
 
def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [ ]:
# Load data (deserialize)
import pickle
with open('ans_dict_inv.pickle', 'rb') as handle:
    ans_dict_inv = pickle.load(handle)

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/model.h5',custom_objects={'KerasLayer':hub.KerasLayer})

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
max_seq_length = 128
def img(file_name ,size = 128):
  img = cv2.imread(file_name)
  img = cv2.resize(img,(size,size))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  return img
 
def predict(filename,question):
  image  = img(file_name)
  plt.imshow(image)
  image = (np.asarray(image).astype('float32') - 127.5) / 127.5
  image = image.reshape(1,128,128,3)
  question = question.lower()
  question = ["[CLS]"] + tokenizer.tokenize(question) + ["[SEP]"]
  id = np.array(get_ids(question, tokenizer, max_seq_length)).reshape(1,128)
  mask = np.array(get_masks(question, max_seq_length)).reshape(1,128)
  segment = np.array(get_segments(question, max_seq_length)).reshape(1,128)
  pred = model.predict([image,id,mask,segment]) 
  return np.argmax(pred)

**YOU CAN RERUN CELLS AFTER IT FOR ANOTHER IMAGE**

In [ ]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
file_name = fn

Saving 16252895930903641793321378138003.jpg to 16252895930903641793321378138003.jpg


**YOU CAN RERUN CELLS AFTER IT FOR ANOTHER QUESTION**

In [ ]:
question = input('ENTER QUESTION :') #like this - what is the color of shirt?
output = predict(fn,question)
answer = ans_dict_inv[output]
print(answer)